# Extract counterexamples from a Z3 proof

In [ ]:
from collections.abc import Callable
from typing import Any, TypeVar

import z3
from z3 import ForAll, Not

V = TypeVar("V", bound=Callable)

def run(fn: V) -> V:
    fn()
    return fn

In [ ]:
# Adapted from
# https://microsoft.github.io/z3guide/programming/Proof%20Logs/#capture-just-quantifier-instantiations

def log_instance(pr: z3.ExprRef, *args: Any) -> None:
    if not z3.is_app(pr) or pr.decl().name() != "inst":
        return
    quant = pr.arg(0)
    if quant.qid() != "synthesis":
        return
    for child in pr.children():
        if not z3.is_app(child) or child.decl().name() != "bind":
            continue
        # Extract the bindings (counterexample values)
        bindings = child.children()
        print(f"Quantifier: {quant}")
        
        # Extract and print the bound variables and their values
        if quant.num_vars() > 0:
            for i in range(quant.num_vars()):
                var_name = quant.var_name(i)
                var_value = bindings[i]
                print(f"Counterexample: {var_name} = {var_value}")


@run
def print_instances() -> None:
    solver = z3.Solver()
    x = z3.Int("x")
    constraint = x * x == 4
    solver.add(ForAll([x], Not(constraint), qid="synthesis"))
    z3.OnClause(solver, log_instance)
    print(solver.check())

In [ ]:
_INSTANCE: list[z3.ExprRef] = []

def save_instance(pr: z3.ExprRef, *args: Any) -> None:
    global _INSTANCE
    if not z3.is_app(pr) or pr.decl().name() != "inst":
        return
    quant = pr.arg(0)
    if quant.qid() != "synthesis":
        return
    for child in pr.children():
        if not z3.is_app(child) or child.decl().name() != "bind":
            continue
        _INSTANCE = child.children()

@run
def extract_instances() -> list[z3.ExprRef]:
    solver = z3.Solver()
    x = z3.Int("x")
    constraint = x * x == 4
    solver.add(ForAll([x], Not(constraint), qid="synthesis"))
    z3.OnClause(solver, save_instance)
    print(solver.check())
    print(_INSTANCE)
    return _INSTANCE